<a href="https://colab.research.google.com/github/Varsha-1403/Prosumer-Energy-Predict/blob/feature-eda-train-forecast/Notebook/ml_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import dask.dataframe as dd

In [4]:

# Load the datasets
#client = pd.read_csv('client.csv')
#gs_prices = pd.read_csv('gas_prices.csv')
#electricity_prices = pd.read_csv('electricity_prices.csv')
train = dd.read_csv('train.csv')
#historical_weather = pd.read_csv('historical_weather.csv')
forecast_weather = dd.read_csv('forecast_weather.csv')
#weather_station_to_county = pd.read_csv('weather_station_to_county_mapping.csv')

In [26]:
# Merge train data with forecast weather data
merged_train_forecast = dd.merge(train, forecast_weather, how='left', left_on=['data_block_id', 'datetime'], right_on=['data_block_id', 'forecast_datetime'])


In [7]:
forecast_weather.head()

,latitude,longitude,origin_datetime,hours_ahead,temperature,dewpoint,cloudcover_high,cloudcover_low,cloudcover_mid,cloudcover_total,10_metre_u_wind_component,10_metre_v_wind_component,data_block_id,forecast_datetime,direct_solar_radiation,surface_solar_radiation_downwards,snowfall,total_precipitation
0,57.6,21.7,2021-09-01 02:00:00,1,15.655786,11.553613,0.904816,0.019714,0.000000,0.905899,-0.411328,-9.106137,1,2021-09-01 03:00:00,0.0,0.0,0.0,0.0
1,57.6,22.2,2021-09-01 02:00:00,1,13.003931,10.689844,0.886322,0.004456,0.000000,0.886658,0.206347,-5.355405,1,2021-09-01 03:00:00,0.0,0.0,0.0,0.0
2,57.6,22.7,2021-09-01 02:00:00,1,14.206567,11.671777,0.729034,0.005615,0.000000,0.730499,1.451587,-7.417905,1,2021-09-01 03:00:00,0.0,0.0,0.0,0.0
3,57.6,23.2,2021-09-01 02:00:00,1,14.844507,12.264917,0.336304,0.074341,0.000626,0.385468,1.090869,-9.163999,1,2021-09-01 03:00:00,0.0,0.0,0.0,0.0
4,57.6,23.7,2021-09-01 02:00:00,1,15.293848,12.458887,0.102875,0.088074,0.000015,0.176590,1.268481,-8.975766,1,2021-09-01 03:00:00,0.0,0.0,0.0,0.0


In [29]:
correlation = merged_train_forecast['temperature'].corr(merged_train_forecast['target'])
print(f"Correlation between Temperature and Target Variable: {correlation}")


Correlation between Temperature and Target Variable: dd.Scalar<corr-0a..., dtype=float64>


/usr/local/lib/python3.10/dist-packages/dask/dataframe/multi.py:1297: UserWarning: Concatenating dataframes with unknown divisions.
We're assuming that the indices of each dataframes are 
 aligned. This assumption is not generally safe.
  warnings.warn(


In [38]:
# Example: Clean up and convert to numeric
merged_train_forecast['temperature'] = pd.to_numeric(merged_train_forecast['temperature'], errors='coerce')


In [3]:
import pandas as pd

# Load datasets
train_data = pd.read_csv('train.csv')
weather_forecast = pd.read_csv('forecast_weather.csv')

# Merge datasets on datetime
merged_data = pd.merge(train_data, weather_forecast[['forecast_datetime', 'temperature']],
                       how='left', left_on='datetime', right_on='forecast_datetime')

# Drop redundant columns and NaN values
merged_data.drop(['forecast_datetime'], axis=1, inplace=True)
merged_data.dropna(inplace=True)

# Now merged_data contains train.csv with temperature data from forecast_weather.csv
# You can proceed with further feature engineering and model training/validation


<ipython-input-3-b61d56da44b3>:4: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv('train.csv')


In [1]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score


In [4]:

# Assuming merged_data contains your prepared dataset
X = merged_data[['temperature', 'county', 'is_business', 'product_type']]  # Features
y = merged_data['target']  # Target variable


In [5]:

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [7]:

# Initialize the RandomForestRegressor
model = RandomForestRegressor(n_estimators=100, random_state=42)

In [ ]:

# Train the model
model.fit(X_train, y_train)

In [ ]:



# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')
